**Experiment 2: DTA, linear regression and subwords**

*Background*: Experiment 2 shows that linear regression with 215 features does not overfit anymore, but the model does not fit the beginning and the end of the timerange. Quadratic polynomial regression would be a possibility, but quadratic polynomial regression performs worse on both training and validation set than linear regression. Polynomial regression with a degree of 5 cannot be computed due to the vast number of polynomial features created by the model.

*Goal*: Determine if it is possible to predict the year in which a text was written using regression.

*Strategies*:

- Use a BPE-transformer to train on subwords (Sennrich2016)

*Relevance*:

- If this experiment works, it is possible to estimate years for corpora that have NA's in this variable.
- Subwords might increase the amount of generalization, and minimize the vocabulary used at the same time.

*Success criteria*:

- Consistent findings over training-, test- and validation set
- predicted year is not more than ten years away from the true year

*Corpora*:

- DTA

*Result*: 

Best parameters: 5 merges, 20 features, 2 merges and 50 features is only slightly worse -> reg_4,5, 6, 7 with 100 features over all merges are pretty close to each other

***Baselines to beat (Exp. 1b)***:

*MSE DTA Train*: 2259.8 | 2715.82

*MSE DTA Val*: 3202.51 | 3321.30

*MSE DTA Test*: 4504.35 | 3135.93

*MSE over GERMANC val*: 3700.60 | 4508.66

*MSE over GERMANC test*: 3565.05 | 4400.24

Setup:

- nbest: 20
- merges: 5

----------------------------------------------------------------------------------------------------------------------

*MSE CLMET Train*: 1346.26

*MSE CLMET Val*: 2727.37

*MSE CLMET Test*: 4315.18

*MSE over ARCHER Val*: 10212.14

*MSE over ARCHER Test*: 9784.28


----------------------------------------------------------------------------------------------------------------------

*MSE ARCHER Train*: 3555.68

*MSE ARCHER Val*: 4843.06

*MSE ARCHER Test*: 4939.51

*MSE over CLMET Val*: 6437126.77

*MSE over CLMET Test*: 6689227.44

----------------------------------------------------------------------------------------------------------------------

*MSE GERMANC Train*: 348.830

*MSE GERMANC Val*: 398.94

*MSE GERMANC Test*: 649.33

*MSE over DTA val*: 8400009.85

*MSE over DTA test*: 21896163.78


In [1]:
import pandas as pd
import numpy as np
import nltk

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest , f_regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import sklearn.utils
from sklearn.preprocessing import FunctionTransformer

from Selfwritten_modules.SubwordTransformer import SubwordTransformer

import re

import eli5

/Users/dianaenggist/opt/anaconda3/envs/R_python/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/dianaenggist/opt/anaconda3/envs/R_python/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#build tokenizer that just substitutes '[' and ']' with ','
def tokenizer_word(doc):
    doc = re.sub('[(\[+)|(\]+)]', '', doc)
    doc = re.split(',', doc)
    return doc

In [3]:
#function for assembling predictions in order to find out how features are weighted

def collect_predictions(dataset, classifier,vectorizer, feature_names, pipeline):
    predictions = eli5.explain_weights_df(classifier,vec=vectorizer, feature_names=feature_names)
    
    predictions = predictions.drop(['target'], axis=1)
    
    
    predictions['YEAR'] = 0
    
    indexes = dataset.index.values
    
    

    for index in indexes:
        
        pred = eli5.explain_prediction_df(classifier, dataset[index], vec=vectorizer, feature_names=feature_names)
        
        source_text = pd.DataFrame([[dataset[index]]])
        
        year_pred = pipeline.predict(source_text[0])
        pred['weight_value'] = pred['weight'] * pred['value']
        pred['instance'] = index
        
        pred = pred.drop(['target','value'], axis=1)
        
    
        pred['YEAR'] = np.round(year_pred[0])
    
        predictions = pd.concat([predictions, pred])
        
    
    
    
    return predictions.dropna()

In [4]:
train_full = pd.read_csv('/Volumes/Korpora/Train/DTA_train_tokenized.csv', sep=';')
val_full = pd.read_csv('/Volumes/Korpora/Val/DTA_val_tokenized.csv', sep=';')
test_full = pd.read_csv('/Volumes/Korpora/Test/DTA_test_tokenized.csv', sep=';')

In [5]:
print('Length train set: ',len(train_full))
print('Length validation set: ', len(val_full))
print('Length test set: ', len(test_full))

Length train set:  899
Length validation set:  225
Length test set:  281


In [6]:
train_x = train_full['Text']
train_y = train_full['Publication_year']

val_x = val_full['Text']
val_y = val_full['Publication_year']

test_x = test_full['Text']
test_y = test_full['Publication_year']

The pipeline takes a lot of time to run through. To determine were the problem is, I'm going to go through the subword transformer stepwise.

In [7]:
subwords = SubwordTransformer(tokenizer=tokenizer_word)

subwords.fit(train_x)

KeyboardInterrupt: 

In [10]:
len(subwords.get_feature_names())

50

So the subword algorithm seems to extract only 10 subwords. This is the number of merges the algorithm performs per default.
It also seems to use about one minute per merge. Maybe it would be useful to extract the k_best pairs each time instead of only the best pair.

The transformation process took the algorithm really long, but I fixed it, now the speed is better.

Update: Parallelizing the extraction of the subwords really helped to gather more features in less time. So I would suggest to rather increase the features extracted at the same time step than the number of merges done, at least when time plays a role.

In [11]:
train_x_transformed = subwords.transform(train_x)

In [7]:
reg_1 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=5, n_best = 50)),
                ('ridge_reg', linear_model.Ridge())
                        ])

In [8]:
reg_1.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=50, number_of_merges=5,
                                    tokenizer=<function tokenizer_word at 0x12214a7a0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [9]:
y_pred_train = reg_1.predict(train_x)
mean_squared_error(train_y, y_pred_train)

1884.2910597026605

In [10]:
y_pred_val = reg_1.predict(val_x)

mean_squared_error(val_y, y_pred_val)

5574.163315600897

In [11]:
reg_2 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=10, n_best = 25)),
                ('ridge_reg', linear_model.Ridge())
                        ])

In [12]:
reg_2.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=25, number_of_merges=10,
                                    tokenizer=<function tokenizer_word at 0x12214a7a0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [13]:
y_pred_train = reg_2.predict(train_x)
mean_squared_error(train_y, y_pred_train)

1868.284018370914

In [14]:
y_pred_val = reg_2.predict(val_x)

mean_squared_error(val_y, y_pred_val)

4819.751780050508

In [15]:
reg_3 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=1, n_best = 250)),
                ('ridge_reg', linear_model.Ridge())
                        ])

In [16]:
reg_3.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=250, number_of_merges=1,
                                    tokenizer=<function tokenizer_word at 0x12214a7a0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [17]:
y_pred_train = reg_3.predict(train_x)
mean_squared_error(train_y, y_pred_train)

1981.3836256602808

In [18]:
y_pred_val = reg_3.predict(val_x)

mean_squared_error(val_y, y_pred_val)

4711.995368986996

The subwords do not beat the baseline yet, but it is very close to it. It seems that the number of merges does increase the performance slightly. We also have more features than the linear regression on which the baseline was trained. So let's play around with the features.

In [19]:
reg_4 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=10, n_best = 10)),
                ('ridge_reg', linear_model.Ridge())
                        ])

In [20]:
reg_4.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=10, number_of_merges=10,
                                    tokenizer=<function tokenizer_word at 0x12214a7a0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [21]:
y_pred_train = reg_4.predict(train_x)
mean_squared_error(train_y, y_pred_train)

2755.1225760989837

In [22]:
y_pred_val = reg_4.predict(val_x)
mean_squared_error(val_y, y_pred_val)

3203.671297418154

100 features with a lot of merges and few words loaded per merge leads to heavily overfitting.

In [23]:
reg_5 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=5, n_best = 20)),
                ('ridge_reg', linear_model.Ridge())])

In [24]:
reg_5.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=20, number_of_merges=5,
                                    tokenizer=<function tokenizer_word at 0x12214a7a0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [25]:
y_pred_train = reg_5.predict(train_x)
mean_squared_error(train_y, y_pred_train)

2715.8187231216534

In [26]:
y_pred_val = reg_5.predict(val_x)
mean_squared_error(val_y, y_pred_val)

3321.302060513491

In [31]:
reg_6 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=2, n_best = 50)),
                ('ridge_reg', linear_model.Ridge())])

In [32]:
reg_6.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=50, number_of_merges=2,
                                    tokenizer=<function tokenizer_word at 0x12214a7a0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [33]:
y_pred_train = reg_6.predict(train_x)
mean_squared_error(train_y, y_pred_train)

2807.3324608991907

In [34]:
y_pred_val = reg_6.predict(val_x)
mean_squared_error(val_y, y_pred_val)

3554.4515758892103

In [35]:
reg_7 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=1, n_best = 100)),
                ('ridge_reg', linear_model.Ridge())])

In [36]:
reg_7.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=100, number_of_merges=1,
                                    tokenizer=<function tokenizer_word at 0x12214a7a0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [37]:
y_pred_train = reg_7.predict(train_x)
mean_squared_error(train_y, y_pred_train)

2786.7084867524163

In [38]:
y_pred_val = reg_7.predict(val_x)
mean_squared_error(val_y, y_pred_val)

3545.195196886467

In [20]:
reg_8 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=2, n_best = 60)),
                ('ridge_reg', linear_model.Ridge())])

In [21]:
reg_8.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=60, number_of_merges=2,
                                    tokenizer=<function tokenizer_word at 0x1250d3b90>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [22]:
y_pred_train = reg_8.predict(train_x)
mean_squared_error(train_y, y_pred_train)

2465.011096075109

In [23]:
y_pred_val = reg_8.predict(val_x)
mean_squared_error(val_y, y_pred_val)

3602.910503824884

In [25]:
reg_9 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=2, n_best = 80)),
                ('ridge_reg', linear_model.Ridge())])

In [26]:
reg_9.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=80, number_of_merges=2,
                                    tokenizer=<function tokenizer_word at 0x1250d3b90>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [27]:
y_pred_train = reg_9.predict(train_x)
mean_squared_error(train_y, y_pred_train)

2222.48922717676

In [28]:
y_pred_val = reg_9.predict(val_x)
mean_squared_error(val_y, y_pred_val)

3667.109164393909

In [29]:
reg_10 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=2, n_best = 70)),
                ('ridge_reg', linear_model.Ridge())])

In [30]:
reg_10.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=70, number_of_merges=2,
                                    tokenizer=<function tokenizer_word at 0x1250d3b90>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [31]:
y_pred_train = reg_10.predict(train_x)
mean_squared_error(train_y, y_pred_train)

2192.9760224052943

In [32]:
y_pred_val = reg_10.predict(val_x)
mean_squared_error(val_y, y_pred_val)

4156.00168293966

In [33]:
reg_11 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=3, n_best = 50)),
                ('ridge_reg', linear_model.Ridge())])

In [34]:
reg_11.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=50, number_of_merges=3,
                                    tokenizer=<function tokenizer_word at 0x1250d3b90>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [35]:
y_pred_train = reg_11.predict(train_x)
mean_squared_error(train_y, y_pred_train)

2154.155532780347

In [36]:
y_pred_val = reg_11.predict(val_x)
mean_squared_error(val_y, y_pred_val)

4410.141749003236

In [37]:
reg_12 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=3, n_best = 30)),
                ('ridge_reg', linear_model.Ridge())])

In [38]:
reg_12.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=30, number_of_merges=3,
                                    tokenizer=<function tokenizer_word at 0x1250d3b90>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [39]:
y_pred_train = reg_12.predict(train_x)
mean_squared_error(train_y, y_pred_train)

2148.1071957613735

In [40]:
y_pred_val = reg_12.predict(val_x)
mean_squared_error(val_y, y_pred_val)

4429.4427212795745

In [8]:
reg_13 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=5, n_best = 60)),
                ('ridge_reg', linear_model.Ridge())
                        ])

In [9]:
reg_13.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=60, number_of_merges=5,
                                    tokenizer=<function tokenizer_word at 0x11f4b77a0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [10]:
y_pred_train = reg_13.predict(train_x)
mean_squared_error(train_y, y_pred_train)

1701.4282076101638

In [11]:
y_pred_val = reg_13.predict(val_x)
mean_squared_error(val_y, y_pred_val)

6828.848691965685

**Try out best settings on different corpora**

In [8]:
reg_5 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=5, n_best = 20)),
                ('ridge_reg', linear_model.Ridge())
                        ])

In [9]:
reg_5.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=20, number_of_merges=5,
                                    tokenizer=<function tokenizer_word at 0x11d17d7a0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [10]:
y_pred_train = reg_5.predict(train_x)
mean_squared_error(train_y, y_pred_train)

2715.8187231216534

In [12]:
y_pred_val = reg_5.predict(val_x)
mean_squared_error(val_y, y_pred_val)

3321.302060513491

In [13]:
y_pred_test = reg_5.predict(test_x)
mean_squared_error(test_y, y_pred_test)

3135.931293691361

In [16]:
features_selected = reg_5['subwords'].get_feature_names()

eli5.explain_weights(reg_5['ridge_reg'],vec=reg_5['subwords'], feature_names=features_selected)

Explanation(estimator="Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,\n      normalize=False, random_state=None, solver='auto', tol=0.001)", description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=True, targets=[TargetExplanation(target='y', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=1778.3498690889428, std=None, value=None), FeatureWeight(feature='men', weight=0.0605012486524709, std=None, value=None), FeatureWeight(feature='dur', weight=0.044271392991742264, std=None, value=None), FeatureWeight(feature='aus', weight=0.03948769413340068, std=None, value=None), FeatureWeight(feature='dem', weight=0.03390043418486171, std=None, value=None), FeatureWeight(feature='ir', weight=0.025632102629888187, std=None, value=None), FeatureWeight(feature='icht', weight=0.02343606729186263, std=None, value=None), FeatureWeight(feature='aͤ', weight=0.020927431923458632, std=None, value=None), FeatureWeight(feature='ma', weight=0.019242514789179925, std=None, value=None)], neg=[FeatureWeight(feature='urch', weight=-0.03993747429797342, std=None, value=None), FeatureWeight(feature='man', weight=-0.032511569488420385, std=None, value=None), FeatureWeight(feature='pf', weight=-0.02879160535124623, std=None, value=None), FeatureWeight(feature='ſſen', weight=-0.026899639941937902, std=None, value=None), FeatureWeight(feature='oͤ', weight=-0.025076931781702633, std=None, value=None), FeatureWeight(feature='och', weight=-0.02426697506597485, std=None, value=None), FeatureWeight(feature='ach', weight=-0.022715713981774504, std=None, value=None), FeatureWeight(feature='ſich', weight=-0.022215503758750064, std=None, value=None), FeatureWeight(feature='ver', weight=-0.021191320467281994, std=None, value=None), FeatureWeight(feature='me', weight=-0.021015060477789924, std=None, value=None), FeatureWeight(feature='des', weight=-0.019421107985842626, std=None, value=None)], pos_remaining=44, neg_remaining=37), proba=None, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [18]:
y_pred_train = pd.DataFrame(y_pred_train, columns=['Predicted_y'])

diff_pred_true_train = pd.concat([y_pred_train, train_y], axis=1)

diff_pred_true_train['Difference'] = diff_pred_true_train.Predicted_y - diff_pred_true_train.Publication_year
    

y_pred_val = pd.DataFrame(y_pred_val, columns=['Predicted_y'])

diff_pred_true_val = pd.concat([y_pred_val, val_y], axis=1)

diff_pred_true_val['Difference'] = diff_pred_true_val.Predicted_y - diff_pred_true_val.Publication_year


y_pred_test = pd.DataFrame(y_pred_test, columns=['Predicted_y'])

diff_pred_true_test = pd.concat([y_pred_test, test_y], axis=1)

diff_pred_true_test['Difference'] = diff_pred_true_test.Predicted_y - diff_pred_true_test.Publication_year

In [20]:
diff_pred_true_train.to_csv('/Volumes/Korpora/Experiment_results/DTA_Exp2_Reg5_Labels_train.csv',sep=';')
diff_pred_true_val.to_csv('/Volumes/Korpora/Experiment_results/DTA_Exp2_Reg5_Labels_val.csv',sep=';')
diff_pred_true_test.to_csv('/Volumes/Korpora/Experiment_results/DTA_Exp2_Reg5_Labels_test.csv',sep=';')

In [22]:
train_details = collect_predictions(train_x, reg_5['ridge_reg'],reg_5['subwords'],features_selected, reg_5)
val_details = collect_predictions(val_x, reg_5['ridge_reg'],reg_5['subwords'],features_selected, reg_5)
test_details = collect_predictions(test_x, reg_5['ridge_reg'],reg_5['subwords'],features_selected, reg_5)

In [23]:
train_details.to_csv('/Volumes/Korpora/Experiment_results/Exp2_Reg5_Train_results_DTA.csv',sep=';')
val_details.to_csv('/Volumes/Korpora/Experiment_results/Exp2_Reg5_Val_results_DTA.csv', sep=';')
test_details.to_csv('/Volumes/Korpora/Experiment_results/Exp2_Reg5_Test_results_DTA.csv', sep=';')

In [4]:
GERMANC_train_full = pd.read_csv('/Volumes/Korpora/Train/GERMANC_train_tokenized.csv', sep=';')
GERMANC_val_full = pd.read_csv('/Volumes/Korpora/Val/GERMANC_val_tokenized.csv', sep=';')
GERMANC_test_full = pd.read_csv('/Volumes/Korpora/Test/GERMANC_test_tokenized.csv', sep=';')

In [5]:
GERMANC_train_full = GERMANC_train_full[(GERMANC_train_full.Year.str.len()== 4) & (GERMANC_train_full.Year.str.isnumeric())]

GERMANC_val_full = GERMANC_val_full[(GERMANC_val_full.Year.str.len()== 4) & (GERMANC_val_full.Year.str.isnumeric())]

GERMANC_test_full = GERMANC_test_full[(GERMANC_test_full.Year.str.len()== 4) & (GERMANC_test_full.Year.str.isnumeric())]

In [6]:
print('Length train set: ',len(GERMANC_train_full))
print('Length validation set: ', len(GERMANC_val_full))
print('Length test set: ', len(GERMANC_test_full))

Length train set:  177
Length validation set:  40
Length test set:  56


In [7]:
GERMANC_train_x = GERMANC_train_full['Text']
GERMANC_train_y = GERMANC_train_full['Year'].astype(int)

GERMANC_val_x = GERMANC_val_full['Text']
GERMANC_val_y = GERMANC_val_full['Year'].astype(int)

GERMANC_test_x = GERMANC_test_full['Text']
GERMANC_test_y = GERMANC_test_full['Year'].astype(int)

In [28]:
y_pred_val = reg_5.predict(GERMANC_val_x)
mean_squared_error(GERMANC_val_y, y_pred_val)

4508.662308304975

In [29]:
y_pred_test = reg_5.predict(GERMANC_test_x)
mean_squared_error(GERMANC_test_y, y_pred_test)

4400.241534553565

In [30]:

y_pred_val = pd.DataFrame(y_pred_val, columns=['Predicted_y'])

diff_pred_true_val = pd.concat([y_pred_val, GERMANC_val_y], axis=1)

diff_pred_true_val['Difference'] = diff_pred_true_val.Predicted_y - diff_pred_true_val.Year


y_pred_test = pd.DataFrame(y_pred_test, columns=['Predicted_y'])

diff_pred_true_test = pd.concat([y_pred_test, GERMANC_test_y], axis=1)

diff_pred_true_test['Difference'] = diff_pred_true_test.Predicted_y - diff_pred_true_test.Year

In [31]:
diff_pred_true_val.to_csv('/Volumes/Korpora/Experiment_results/DTA_over_GERMANC_Exp2_Reg5_Labels_val.csv',sep=';')
diff_pred_true_test.to_csv('/Volumes/Korpora/Experiment_results/DTA_over_GERMANC_Exp2_Reg5_Labels_test.csv',sep=';')

In [37]:
val_details = collect_predictions(GERMANC_val_x, reg_5['ridge_reg'],reg_5['subwords'],features_selected, reg_5)
test_details = collect_predictions(GERMANC_test_x, reg_5['ridge_reg'],reg_5['subwords'],features_selected, reg_5)

In [38]:
val_details.to_csv('/Volumes/Korpora/DTA_over_GERMANC_Exp2_Reg5_Val_results.csv', sep=';')
test_details.to_csv('/Volumes/Korpora/DTA_over_GERMANC_Exp2_Reg5_Test_results.csv', sep=';')

**GERMANC**

Ratio documents-features DTA: 11%
Ratio features-n_best: 20%
Ratio features-merges: 5%
Ratio n_best-merges DTA: 25%

For GERMANC: 20 features total

In [10]:
reg_5 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=5, n_best = 4)),
                ('ridge_reg', linear_model.Ridge())
                        ])

In [11]:
reg_5.fit(GERMANC_train_x, GERMANC_train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=4, number_of_merges=5,
                                    tokenizer=<function tokenizer_word at 0x124716710>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [12]:
y_pred_train = reg_5.predict(GERMANC_train_x)
mean_squared_error(GERMANC_train_y, y_pred_train)

1086.2656667176987

In [13]:
y_pred_val = reg_5.predict(GERMANC_val_x)
mean_squared_error(GERMANC_val_y, y_pred_val)

2016.0332127419213

In [14]:
reg_5 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=1, n_best = 20)),
                ('ridge_reg', linear_model.Ridge())
                        ])

In [15]:
reg_5.fit(GERMANC_train_x, GERMANC_train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=20, number_of_merges=1,
                                    tokenizer=<function tokenizer_word at 0x124716710>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [16]:
y_pred_train = reg_5.predict(GERMANC_train_x)
mean_squared_error(GERMANC_train_y, y_pred_train)

1198.2428288646931

In [17]:
y_pred_val = reg_5.predict(GERMANC_val_x)
mean_squared_error(GERMANC_val_y, y_pred_val)

1825.41987296582

In [18]:
reg_5 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=2, n_best = 20)),
                ('ridge_reg', linear_model.Ridge())
                        ])

In [19]:
reg_5.fit(GERMANC_train_x, GERMANC_train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=20, number_of_merges=2,
                                    tokenizer=<function tokenizer_word at 0x124716710>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [20]:
y_pred_train = reg_5.predict(GERMANC_train_x)
mean_squared_error(GERMANC_train_y, y_pred_train)

698.6381180160598

In [21]:
y_pred_val = reg_5.predict(GERMANC_val_x)
mean_squared_error(GERMANC_val_y, y_pred_val)

1824.3509233089412

In [22]:
reg_5 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=1, n_best = 10)),
                ('ridge_reg', linear_model.Ridge())
                        ])

In [23]:
reg_5.fit(GERMANC_train_x, GERMANC_train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=10, number_of_merges=1,
                                    tokenizer=<function tokenizer_word at 0x124716710>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [24]:
y_pred_train = reg_5.predict(GERMANC_train_x)
mean_squared_error(GERMANC_train_y, y_pred_train)

1510.5802932520933

In [25]:
y_pred_val = reg_5.predict(GERMANC_val_x)
mean_squared_error(GERMANC_val_y, y_pred_val)

2176.073758107097

In [35]:
reg_5 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=1, n_best = 200)),
                ('ridge_reg', linear_model.Ridge())
                        ])

In [37]:
reg_5.fit(GERMANC_train_x, GERMANC_train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=200, number_of_merges=1,
                                    tokenizer=<function tokenizer_word at 0x11f1b17a0>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [38]:
y_pred_train = reg_5.predict(GERMANC_train_x)
mean_squared_error(GERMANC_train_y, y_pred_train)

0.001291437481472235

In [39]:
y_pred_val = reg_5.predict(GERMANC_val_x)
mean_squared_error(GERMANC_val_y, y_pred_val)

4878.7648829998125

In [27]:
reg_5 = Pipeline([('subwords', SubwordTransformer(tokenizer=tokenizer_word, number_of_merges=1, n_best = 1)),
                ('ridge_reg', linear_model.Ridge())
                        ])

In [28]:
reg_5.fit(GERMANC_train_x, GERMANC_train_y)

Pipeline(memory=None,
         steps=[('subwords',
                 SubwordTransformer(n_best=1, number_of_merges=1,
                                    tokenizer=<function tokenizer_word at 0x124716710>)),
                ('ridge_reg',
                 Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [29]:
y_pred_train = reg_5.predict(GERMANC_train_x)
mean_squared_error(GERMANC_train_y, y_pred_train)

1791.686917033101

In [30]:
y_pred_val = reg_5.predict(GERMANC_val_x)
mean_squared_error(GERMANC_val_y, y_pred_val)

2283.4661576535887

In [31]:
features_selected = reg_5['subwords'].get_feature_names()

eli5.explain_weights(reg_5['ridge_reg'],vec=reg_5['subwords'], feature_names=features_selected)

Weight?,Feature
+1699.972,<BIAS>
+0.061,en


In [32]:
features_selected = reg_5['subwords'].get_feature_names()
len(features_selected)

1